In [ ]:
%load_ext autoreload
%autoreload 2
from hig_data.coco2 import COCOStuffGraphPrecomputedDataset
from hig_data.utils import DataLoader
import torch
# new_coco_val2017_hig.h5
# aug_coco_train2017_hig.h5
path = '/home/rfsm2/rds/hpc-work/datasets/coco/precomputed_coco_val_paths.json'
dataset = COCOStuffGraphPrecomputedDataset(path, swapped=False, )
from hig_data.utils import DataLoader
dls = DataLoader(dataset, batch_size=8, subsample=False)

In [ ]:

import torch
graph_batch = next(iter(dls))
ckpt_path = '/home/rfsm2/rds/hpc-work/training_runs/edm2_pretrained_med/backup_nov_26/edm2-coco256-m_Nov_26_hr_11training-state-0033554.pt'
ckpt = torch.load(ckpt_path)

In [ ]:
from training.networks_edm2_hignn_control import Precond
precond = Precond(64, 4, gnn_metadata = graph_batch.metadata(), model_channels=256, label_dim=768)
precond.load_state_dict(ckpt['net'], strict=False)

In [ ]:
np.log(0.73)

In [ ]:
import os
import re
import pickle


run_dir = '/home/rfsm2/rds/hpc-work/training_runs/edm2_pretrained_xs_uncond/pretrained'
pattern=r'edm2-img512.*(\d+).pkl'
verbose=True
fnames = [entry.name for entry in os.scandir(run_dir) if entry.is_file() and re.fullmatch(pattern, entry.name)]
print(fnames)
pkl_path = os.path.join(run_dir, max(fnames, key=lambda x: float(re.fullmatch(pattern, x).group(1))))

if verbose:
    print(f'Loading from {pkl_path} ... ', end='', flush=True)
with open(pkl_path, 'rb') as f:
    data = pickle.load(f, fix_imports=True, encoding="bytes")

pretrained = data.ema

# # Extract the model state dict from the .pkl data
# model_state_dict = data.ema.state_dict()

# # filters out anything not in model_state + mismatched sizes + emb gain parameter (we want 0 cond emb gain at start of training)
# model_state_dict = {k: v for k, v in model_state_dict.items() if 'emb_label' not in k}

# # Non-strictly load the state dict into the target model
# pretrained_uncond = Precond(64, 4, gnn_metadata = dataset[0].metadata(), model_channels=128, label_dim=0)
# pretrained_uncond.load_state_dict(model_state_dict, strict=False)


In [ ]:
import torch
from training.networks_edm2_hignn_control import Precond
graph_batch = next(iter(dls))
ckpt_path = '/home/rfsm2/rds/hpc-work/training_runs/edm2_pretrained_med/backup_nov_26/edm2-coco256-m_Nov_26_hr_11training-state-0033554.pt'
ckpt = torch.load(ckpt_path)
control_net = Precond(64, 4, gnn_metadata = dataset[0].metadata(), model_channels=256, label_dim=768)
control_net.load_state_dict(ckpt['net'], strict=False)

In [ ]:
from generate_images import edm_sampler


sample_shape = (8, 4, 64, 64)
noise = torch.randn(sample_shape, device='cuda')
sampled = edm_sampler(net=control_net.to(noise.device), noise=noise, graph=graph_batch, num_steps=64,) # samp
# sampled = edm_sampler(net=control_net.to(noise.device), gnet=pretrained.to(noise.device), noise=noise, graph=graph_batch, num_steps=64, guidance=1.4) # sample images from noise and graph batch

In [ ]:
from hig_data.visualisation import plot_array_images
from training.encoders import StabilityVAEEncoder
# graph_batch = next(iter(dls))
vae = StabilityVAEEncoder(batch_size=8)
# print(graph_batch.image.shape, sampled.shape)
ground_truth = vae.decode(vae.encode_latents(graph_batch.image.to('cuda')))
sampled_pixels = vae.decode(sampled.to('cuda'))
plot_array_images(sampled_pixels.cpu()) 
plot_array_images(ground_truth.cpu()) 

In [ ]:
pretrained_uncond.unet.init_control_weights()

In [ ]:
from generate_images import edm_sampler

graph_batch = next(iter(dls))
sample_shape = (8, 4, 64, 64)
noise = torch.randn(sample_shape, device='cuda')
# sampled = edm_sampler(net=precond.to(noise.device), gnet=pretrained_uncond.to(noise.device), noise=noise, graph=graph_batch, num_steps=64, guidance=1.1) # sample images from noise and graph batch
sampled = edm_sampler(net=pretrained_uncond.to(noise.device), noise=noise, graph=graph_batch, num_steps=64,) # samp
# sampled = edm_sampler(net=data.ema.to(noise.device), noise=noise, graph=graph_batch, num_steps=64,) # samp

In [ ]:
from hig_data.visualisation import plot_array_images
from training.encoders import StabilityVAEEncoder
# graph_batch = next(iter(dls))
vae = StabilityVAEEncoder(batch_size=8)
# print(graph_batch.image.shape, sampled.shape)
# ground_truth = vae.decode(vae.encode_latents(graph_batch.image.to('cuda')))
sampled_pixels = vae.decode(sampled.to('cuda'))
plot_array_images(sampled_pixels.cpu()) 
# plot_array_images(ground_truth.cpu()) 

In [ ]:
from generate_images import edm_sampler

graph_batch = next(iter(dls))
sample_shape = (8, 4, 64, 64)
noise = torch.randn(sample_shape, device='cuda')
# sampled = edm_sampler(net=precond.to(noise.device), gnet=pretrained_uncond.to(noise.device), noise=noise, graph=graph_batch, num_steps=64, guidance=1.1) # sample images from noise and graph batch
# sampled = edm_sampler(net=pretrained_uncond.to(noise.device), noise=noise, graph=None, num_steps=64,) # samp
sampled = edm_sampler(net=pretrained_uncond.to(noise.device), noise=noise, graph=graph_batch, num_steps=64,) # samp

In [ ]:
from hig_data.visualisation import plot_array_images
from training.encoders import StabilityVAEEncoder
# graph_batch = next(iter(dls))
vae = StabilityVAEEncoder(batch_size=8)
# print(graph_batch.image.shape, sampled.shape)
# ground_truth = vae.decode(vae.encode_latents(graph_batch.image.to('cuda')))
sampled_pixels = vae.decode(sampled.to('cuda'))
plot_array_images(sampled_pixels.cpu()) 
# plot_array_images(ground_truth.cpu()) 

In [ ]:
from torch.optim import Adam
optimizer_kwargs = dict(betas=(0.9, 0.99), lr=1e-2)
net = pretrained_uncond
new_params = [param for name, param in net.named_parameters() if 'hignn' in name]
pretrained_params = [param for name, param in net.named_parameters() if 'hignn' not in name]

optimizer = Adam(params=[{"params": pretrained_params, "lr": 1e-5}, {"params": new_params,}], **optimizer_kwargs)
optimizer.param_groups[0]['lr']